In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign, expand
from IPython.display import display, HTML, Math

In [2]:
def large(expr):
  latex_code = f"\\Large {{ {latex(expr)} }}"
  display(Math(latex_code))

In [3]:
delta_b_negative = False;
discrete = False;
simp = True;

In [4]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'l', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, l_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, l_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

in the contract, where Delta_b is negative:

-Delta_b * s_s <= Delta_s * s_b

implying for positive Delta_b:

Delta_b * s_s <= Delta_s * s_b

which maximizes value for the trader if:

In [5]:
if delta_b_negative:
  a0_eq_Delta = Eq(-Delta_b * s_s, Delta_s * s_b)
else:
  a0_eq_Delta = Eq(Delta_b * s_s, Delta_s * s_b)
a0_eq_Delta

Eq(Delta_b*s_s, Delta_s*s_b)

In [6]:
if delta_b_negative:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b + Delta_b) * s_s + (l_s + Delta_s) * s_b)
else:
  a0_eq_Value = Eq(l_b * s_s + l_s * s_b, (l_b - Delta_b) * s_s + (l_s + Delta_s) * s_b)
a0_eq_Value

Eq(l_b*s_s + l_s*s_b, s_b*(Delta_s + l_s) + s_s*(-Delta_b + l_b))

In [7]:
assert(
  solveset(a0_eq_Value, Delta_b).simplify() == solveset(a0_eq_Delta, Delta_b),
);
assert(
  solveset(a0_eq_Value, Delta_s).simplify() == solveset(a0_eq_Delta, Delta_s),
);
a0_eq = a0_eq_Delta;

In [8]:
# this seems to be derived from spotByDelta
def deltaBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = (s - l * w) / w # at this point, this value is negative for buying and positive for selling
  # rounding towards zero in all cases, as we are interested in the maximum delta-capacity of a given spot price
  if asset == 'b':
    if delta_b_negative:
      if discrete:
        f = ceiling(f)
    else:
      if discrete:
        f = floor(-f)
      else:
        f = -f

  else:
    if discrete:
      f = floor(f)

  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaBySpot['b']))
display(Eq(Delta_s, deltaBySpot['s']))

Eq(Delta_b, l_b - s_b/w_b)

Eq(Delta_s, -l_s + s_s/w_s)

In [9]:
solveset(Eq(Delta_b, deltaBySpot["b"]), s_b);

{-w_b*(Delta_b - l_b)}

In [10]:
solveset(Eq(Delta_s, deltaBySpot["s"]), s_s);

{w_s*(Delta_s + l_s)}

### logic behind spotByDelta

- we need to pick a spot price that is worse than the amm price = w * l
- for uninverted prices, worse means larger for buying and lower for selling
- since prices are inverted, worse means smaller for buying and larger for
  selling
- this must hold true after the trade as well, so it must also be larger than
  w * (l + Delta) for selling and smaller than w * (l - Delta) for buying
- (assuming Delta_b is represented positively, otherwise it's larger (selling)
  resp. smaller (buying) than w * (l + Delta) for both)

In [11]:
def spotByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  if (not delta_b_negative) and asset == 'b':
    f = w * (l - Delta)
  else:
    f = w * (l + Delta)
  if simp:
    f = simplify(f)
  return f

spotByDelta = {asset: spotByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(s_b, spotByDelta['b']))
display(Eq(s_s, spotByDelta['s']))

Eq(s_b, w_b*(-Delta_b + l_b))

Eq(s_s, w_s*(Delta_s + l_s))

In [12]:
assert(Eq(spotByDelta['b'].subs({Delta_b: deltaBySpot['b']}), s_b))
assert(Eq(spotByDelta['s'].subs({Delta_s: deltaBySpot['s']}), s_s))
if discrete:
  display(deltaBySpot['b'].subs({s_b: spotByDelta['b']}))
  display(deltaBySpot['s'].subs({s_s: spotByDelta['s']}))
else:
  assert(Eq(deltaBySpot['b'].subs({s_b: spotByDelta['b']}), Delta_b))
  assert(Eq(deltaBySpot['s'].subs({s_s: spotByDelta['s']}), Delta_s))

In [13]:
a0_eq.subs({ s_b: spotByDelta["b"], s_s: spotByDelta["s"] }).simplify();

Eq(Delta_b*w_s*(Delta_s + l_s), Delta_s*w_b*(-Delta_b + l_b))

In [14]:
def deltaByDelta_(asset, s, l, v, b, w, j, e, Delta, a, min):
  # if discrete:
  #   if asset == 'b' and delta_b_negative:
  #     domain = S.Integers
  #   else:
  #     domain = S.Naturals
  # else:
  #   domain = S.Reals
  # above takes more than 11 minutes, skipping for now
  f = solveset(a0_eq.subs({s_b: spotByDelta['b'], s_s: spotByDelta['s']}), Delta)#, domain=domain)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if discrete:
    if asset == 'b':
      f = floor(f)
    else:
      f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaByDelta = {asset: deltaByDelta_(asset, *all_symbols[asset]) for asset in assets}
display(Eq(Delta_b, deltaByDelta['b']))
display(Eq(Delta_s, deltaByDelta['s']))

Eq(Delta_b, Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))

Eq(Delta_s, -Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))

In [15]:
print(Eq(Delta_b, deltaByDelta["b"]));
print(Eq(Delta_s, deltaByDelta["s"]));

Eq(Delta_b, Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s))
Eq(Delta_s, -Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b))


### note:

- the above is undefined/negative (which reduces to undefined considering the
  meaning) for some values.
- when we represent Delta_b negatively, this issue appears to vanish, but
  actually doesn't when pondering the sign of Delta_b

In [16]:
denominator_Delta_b = deltaByDelta["b"].args[-1].args[0];
display(denominator_Delta_b);
ndef_Delta_s = solveset(denominator_Delta_b, Delta_s);
assert(len(ndef_Delta_s) == 1);
ndef_Delta_s = list(ndef_Delta_s)[0];
ndef_Delta_s;

Delta_s*w_b + Delta_s*w_s + l_s*w_s

-l_s*w_s/(w_b + w_s)

In [17]:
denominator_Delta_s = deltaByDelta["s"].args[-1].args[0];
display(denominator_Delta_s);
ndef_Delta_b = solveset(denominator_Delta_s, Delta_b);
assert(len(ndef_Delta_b) == 1);
ndef_Delta_b = list(ndef_Delta_b)[0];
ndef_Delta_b;

Delta_b*w_b + Delta_b*w_s - l_b*w_b

l_b*w_b/(w_b + w_s)

In [18]:
display(solveset(a0_eq, Delta_s));
display(solveset(a0_eq, Delta_b));

{Delta_b*s_s/s_b}

{Delta_s*s_b/s_s}

given that we can lower s_b arbitrarily (up to a lower bound of 1) and increase
s_s arbitrarily, we should always be able to get a positive Delta_s

given that we can get some negative Delta_s, does this imply an exploit?

- in the onchain-code, the swap prices can never be negative, as they are
  computed there from exponents
- likewise, if Delta_b is negative (as it should be there), -Delta_b * s_s is
  positive
- -> negative Delta_s would violate -Delta_b * s_s <= Delta_s * s_b (for
  negative Delta_b)

-> no exploit onchain (except potentially negating both deltas, which we examine
further below)

### how do we adjust the equations for the spot prices accordingly, such that the user still get's the best effective price?

And after we found an equation, how does it reconcile with the one we got
before?

In [19]:
lowest_spot = (s_s / s_b).subs({s_s: spotByDelta['s'], s_b: spotByDelta['b']})#.simplify()
lowest_spot

w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

lowest_spot:

- best (lowest) uninverted spot price the user can get (with highest inverted
  buying price and lowest inverted selling price)
- evidently always positive

In [20]:
Ge(Delta_s, (Delta_b * lowest_spot))#.simplify()

Delta_s >= Delta_b*w_s*(Delta_s + l_s)/(w_b*(-Delta_b + l_b))

above:

- as Delta_b is positive (in the respective representation) and spot is
  positive, Delta_s should always be positive
- this however is just a rearrangement of what we already have, resulting in the
  same paradox

### adding extra price multiplier

given that for Delta_b -> l_b * w_b / (w_b + w_s) Delta_s goes towards infinity
in the equation derived from choosing best possible prices, we add an additional
multiplier >= 1 to reflect worsening of the pair spot price for situations where
Delta_b >= l_b * w_b / (w_b + w_s). This can then be translated in an arbitrary
combination of worsening of s_b and s_s.

In [21]:
ndef_Delta_b;

l_b*w_b/(w_b + w_s)

In [22]:
m = symbols("m", nonnegative = True, real = True);
m_Delta_s = Delta_b * lowest_spot * (1 + m);
m_Delta_s;

Delta_b*w_s*(Delta_s + l_s)*(m + 1)/(w_b*(-Delta_b + l_b))

In [23]:
m_DeltaByDelta_b = solveset(Eq(m_Delta_s, Delta_s), Delta_b);
display(m_DeltaByDelta_b);
m_DeltaByDelta_b = m_DeltaByDelta_b.args[0].args[0];
m_DeltaByDelta_b;

Complement({Delta_s*l_b*w_b/(Delta_s*m*w_s + Delta_s*w_b + Delta_s*w_s + l_s*m*w_s + l_s*w_s)}, {l_b})

Delta_s*l_b*w_b/(Delta_s*m*w_s + Delta_s*w_b + Delta_s*w_s + l_s*m*w_s + l_s*w_s)

In [24]:
m_DeltaByDelta_s = solveset(Eq(m_Delta_s, Delta_s), Delta_s);
display(m_DeltaByDelta_s);
m_DeltaByDelta_s = m_DeltaByDelta_s.args[0];
m_DeltaByDelta_s;

{-Delta_b*l_s*w_s*(m + 1)/(Delta_b*m*w_s + Delta_b*w_b + Delta_b*w_s - l_b*w_b)}

-Delta_b*l_s*w_s*(m + 1)/(Delta_b*m*w_s + Delta_b*w_b + Delta_b*w_s - l_b*w_b)

In [25]:
m_DeltaByDelta_s.subs({ Delta_b: ndef_Delta_b });

-l_b*l_s*w_b*w_s*(m + 1)/((w_b + w_s)*(l_b*m*w_b*w_s/(w_b + w_s) + l_b*w_b**2/(w_b + w_s) + l_b*w_b*w_s/(w_b + w_s) - l_b*w_b))

In [26]:
m_DeltaByDelta_s.subs({ Delta_b: ndef_Delta_b }).simplify();

-l_s - l_s/m

this is always negative - why didn't it work?

-> we are at least making the mistake of effectively increasing Delta_s on the
lhs, without adjusting s_s accordingly

(additive also didn't work)

In [27]:
Ge(Delta_s, (Delta_b * lowest_spot)).subs({Delta_b: ndef_Delta_b})#.simplify()

Delta_s >= l_b*w_s*(Delta_s + l_s)/((w_b + w_s)*(-l_b*w_b/(w_b + w_s) + l_b))

In [28]:
(Delta_b * lowest_spot).subs({ Delta_b: ndef_Delta_b }).simplify();

Delta_s + l_s

In [29]:
(Delta_b * lowest_spot * (m + 1)).subs({ Delta_b: ndef_Delta_b }).simplify();

(Delta_s + l_s)*(m + 1)

we can see that the undefined Delta_b results in a situation where Delta_s >=
smth gt Delta_s

let's try correcting s_s first:

In [30]:
display(m_Delta_s);
m_Delta_s_2 = m_Delta_s.subs({ Delta_s: Delta_s / (1 + m) }).simplify();
m_Delta_s_2;

Delta_b*w_s*(Delta_s + l_s)*(m + 1)/(w_b*(-Delta_b + l_b))

-Delta_b*w_s*(Delta_s + l_s*(m + 1))/(w_b*(Delta_b - l_b))

In [31]:
m_DeltaByDelta_s_2 = solveset(Eq(m_Delta_s_2, Delta_s), Delta_s);
display(m_DeltaByDelta_s_2);
m_DeltaByDelta_s_2 = m_DeltaByDelta_s_2.args[0];
m_DeltaByDelta_s_2;

{-Delta_b*l_s*w_s*(m + 1)/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)}

-Delta_b*l_s*w_s*(m + 1)/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)

In [32]:
m_DeltaByDelta_s_2.subs({ Delta_b: ndef_Delta_b }).simplify();

zoo

### we conclude

that the unsolveablity of the issue boils down to the simple fact that:

- as Delta_b approaches l_b * w_b / (w_b + w_s), Delta_s approaches infinity
- this means the effective price Delta_s / Delta_b approaches infinity
- if we were able to remedy that, we would be able to reduce the effective price
  at least at some points, which should not be allowed

-> let's just note the additional constraint

TODO: Add this check to the offchain/frontend such that the pool creator knows
when some liquidity can never be bought. This might however require some
additional analysis for the multi-asset-case. Also note that each swap reduces
l_b, so maybe this is a non-issue after all

(below is just some check that fits nowhere really)

In [33]:
solveset(Eq(Delta_s, deltaByDelta["s"]), Delta_b);

Complement({Delta_s*l_b*w_b/(Delta_s*w_b + Delta_s*w_s + l_s*w_s)}, {l_b*w_b/(w_b + w_s)})

In [34]:
solveset(Eq(Delta_b, deltaByDelta["b"]), Delta_s);

Complement({-Delta_b*l_s*w_s/(Delta_b*w_b + Delta_b*w_s - l_b*w_b)}, {-l_s*w_s/(w_b + w_s)})

### what happens if an attacker simply removes the "selling" asset and adds the "buying" one instead of the other way around?

- onchain equation: -Delta_b * s_s <= Delta_s * s_b
- representing Delta_b positively: Delta_b * s_s <= Delta_s * s_b
- => effectivePrice = Delta_s / Delta_b >= s_s / s_b = spotPrice
- spotPrice conditions:
  - prior: s_b <= w_b * l_b, s_s >= w_s * l_s
    - => spotPrice >= w_s * l_s / (w_b * l_b)
  - posterior: s_b <= w_b * (l_b - Delta_b), s_s >= w_s * (l_s + Delta_s)
    - => spotPrice >= w_s * (l_s + Delta_s) / (w_b * (l_b - Delta_b))

if Delta_b is positive and Delta_s is negative:

- the equation is not violated per se
- the effective price stays the same as the negations cancel out
- however, the condition for the posterior spotPrice changes:
  - s_b <= w_b * (l_b + Delta_b), s_s >= w_s * (l_s - Delta_s)
    - => spotPrice >= w_s * (l_s - Delta_s) / (w_b * (l_b + Delta_b))
      - --> this constitutes an exploit!

### fix

- require correct signs for balance-changes
- we can also drop the pre-swap price requirement, as it's subsumed by the
  post-swap one

# worst case rounding analysis

- rounding the Deltas cannot do worse than increasing Delta_s by 1 and
  decreasing Delta_b by 1
- spotByDelta has no divisions and is therefore unaffected by rounding
- we are not accounting yet for the rounding effect of having to express spots
  via exponents, not here or above (TODO, TODO)

In [35]:
eff_best = Delta_s / Delta_b # effective price we want to minimize
display(eff_best)
eff_worst = (Delta_s + 1) / (Delta_b - 1) # noninclusive worst case resulting from rounding Deltas
eff_worst

Delta_s/Delta_b

(Delta_s + 1)/(Delta_b - 1)

In [36]:
large(eff_best.subs({ Delta_b: deltaByDelta["b"] }).simplify());
large(eff_best.subs({ Delta_s: deltaByDelta["s"] }).simplify());

<IPython.core.display.Math object>

<IPython.core.display.Math object>

-> simply minimized by minimizing Deltas

In [37]:
eff_worst_Delta_s = eff_worst.subs({ Delta_b: deltaByDelta["b"] });
large(eff_worst_Delta_s);
eff_worst_Delta_s = eff_worst_Delta_s.simplify();
large(eff_worst_Delta_s);

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [38]:
large(expand(eff_worst_Delta_s).simplify());

<IPython.core.display.Math object>

In [39]:
numerator = Delta_s ** 2 * (w_b + w_s) + Delta_s * (l_s * w_s + w_b + w_s) +
  l_s * w_s;
denominator = Delta_s * (l_b * w_b - w_b - w_s) - l_s * w_s;
f = numerator / denominator;
large(f);
Eq(f, eff_worst_Delta_s).simplify();

<IPython.core.display.Math object>

True

In [40]:
a, b, c, d_pos = symbols("a b c d_pos", positive = True, integer = True);

a_subs = w_b + w_s;
b_subs = l_s * w_s + w_b + w_s;
c_subs = l_s * w_s;
d_subs = l_b * w_b - w_b - w_s;

f1 = f.subs({ a_subs: a, b_subs: b, c_subs: c, d_subs: d_pos });
display(f1);

(Delta_s**2*a + Delta_s*b + c)/(Delta_s*d_pos - c)

In [41]:
d_f1 = f1.diff(Delta_s).simplify();
display(d_f1);

(-d_pos*(Delta_s**2*a + Delta_s*b + c) + (2*Delta_s*a + b)*(Delta_s*d_pos - c))/(Delta_s*d_pos - c)**2

In [42]:
sols_f1 = solveset(d_f1, Delta_s);
display(sols_f1);

{c/d_pos - sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos), c/d_pos + sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos)}

In [43]:
sol1 = Eq(Delta_s, sols_f1.args[0]);
display(sol1);
print("\t-> everything positive -> solution positive -> VALID");
sol2 = Eq(Delta_s, sols_f1.args[1]);
display(sol2);
print(
  "\t-> needs to be investigated further (spoiler: it'll turn out to be a maximum anyways, so we can skip it)",
);

Eq(Delta_s, c/d_pos + sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos))

	-> everything positive -> solution positive -> VALID


Eq(Delta_s, c/d_pos - sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos))

	-> needs to be investigated further (spoiler: it'll turn out to be a maximum anyways, so we can skip it)


What happens if d is zero or negative?

- zero d:

In [44]:
display(Eq(d_subs, 0))
display(f) # recall: this is our reformulation of the effective price by Delta_s, worst case scenario
display(f.subs({d_subs: 0}).simplify())
display(f.subs({d_subs: 0}).simplify().expand())

Eq(l_b*w_b - w_b - w_s, 0)

(Delta_s**2*(w_b + w_s) + Delta_s*(l_s*w_s + w_b + w_s) + l_s*w_s)/(Delta_s*(l_b*w_b - w_b - w_s) - l_s*w_s)

(Delta_s**2*(-w_b - w_s) - Delta_s*(l_s*w_s + w_b + w_s) - l_s*w_s)/(l_s*w_s)

-Delta_s**2*w_b/(l_s*w_s) - Delta_s**2/l_s - Delta_s - Delta_s*w_b/(l_s*w_s) - Delta_s/l_s - 1

-> given that all summands for the effective price are negative, this solution
is indeed invalid

- negative d:

In [45]:
display(Lt(d_subs, 0));

l_b*w_b - w_b - w_s < 0

-> denominator is negative, numerator is positive -> effective price is negative
-> invalid

=> there are no solutions besides the ones for d_pos

In [46]:
display(Gt(d_subs, 0));
display(Gt(l_b - 1, w_s / w_b));

l_b*w_b - w_b - w_s > 0

l_b - 1 > w_s/w_b

### analysing second derivative to determine minimum-status of solutions:

In [47]:
dd_f1 = d_f1.diff(Delta_s).simplify();
large(dd_f1);

<IPython.core.display.Math object>

In [48]:
dd_sol1 = dd_f1.subs({Delta_s: sol1.rhs}).simplify()
large(dd_sol1)
if Eq(dd_sol1, 0) == False: print("-> nonzero -> extremum")
if (dd_sol1.is_positive == True): print("-> positive -> minimum")
dd_sol2 = dd_f1.subs({Delta_s: sol2.rhs}).simplify()
large(dd_sol2)
if Eq(dd_sol2, 0) == False: print("-> nonzero -> extremum")
if (dd_sol2.is_negative == True): print("-> negative -> maximum")

<IPython.core.display.Math object>

-> nonzero -> extremum
-> positive -> minimum


<IPython.core.display.Math object>

-> nonzero -> extremum
-> negative -> maximum


### investigating limits:

In [49]:
display(f1.subs(Delta_s, 0));
display(limit(f1, Delta_s, oo));
display(limit(f1, Delta_s, -oo));

-1

oo

-oo

In [50]:
f1_zero = solveset(f1, Delta_s);
display(f1_zero);
print("crosses x-axis at:");
display(f1_zero.args[0].args[0].simplify());
display(
  f1_zero.args[0].args[0].subs({
    a: a_subs,
    b: b_subs,
    c: c_subs,
    d_pos: d_subs,
  }).simplify(),
);
print("and:");
display(f1_zero.args[0].args[1].simplify());
display(
  f1_zero.args[0].args[1].subs({
    a: a_subs,
    b: b_subs,
    c: c_subs,
    d_pos: d_subs,
  }).simplify(),
);

Complement({-b/(2*a) - sqrt(-4*a*c + b**2)/(2*a), -b/(2*a) + sqrt(-4*a*c + b**2)/(2*a)}, {c/d_pos})

crosses x-axis at:


(-b - sqrt(-4*a*c + b**2))/(2*a)

(-l_s*w_s - w_b - w_s - sqrt(-4*l_s*w_s*(w_b + w_s) + (l_s*w_s + w_b + w_s)**2))/(2*(w_b + w_s))

and:


(-b + sqrt(-4*a*c + b**2))/(2*a)

(-l_s*w_s - w_b - w_s + sqrt(-4*l_s*w_s*(w_b + w_s) + (l_s*w_s + w_b + w_s)**2))/(2*(w_b + w_s))

In [51]:
print("singularity at:");
f1_ndef = f1_zero.args[1].args[0];
display(f1_ndef);
display(f1_ndef.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs }));
print("singularity-limit from below:");
display(limit(f1, Delta_s, f1_ndef, dir = "-"));
print("singularity-limit from above:");
display(limit(f1, Delta_s, f1_ndef, dir = "+"));

singularity at:


c/d_pos

l_s*w_s/(l_b*w_b - w_b - w_s)

singularity-limit from below:


-oo

singularity-limit from above:


oo

In [52]:
print("maximum at:");
display(sol2);
display(
  sol2.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs }).simplify(),
);

maximum at:


Eq(Delta_s, c/d_pos - sqrt(c)*sqrt(a*c + b*d_pos + d_pos**2)/(sqrt(a)*d_pos))

Eq(Delta_s, (-sqrt(l_b)*sqrt(l_s)*sqrt(w_b)*sqrt(w_s)*sqrt(l_b*w_b + l_s*w_s - w_b - w_s) + l_s*w_s*sqrt(w_b + w_s))/(sqrt(w_b + w_s)*(l_b*w_b - w_b - w_s)))

-> we have a function that

- comes from -oo
- increases
- crosses the x-axis from below at some negative x (see equation above, which
  has only negative summands)
- reaches a maximum (which needs to happen between the two crossings of the
  x-axis)
- decreases
- crosses the x-axis from above at some negative x (we know this because we know
  that it crosses the y-axis at -1)
- crosses the y-axis at -1 (where the x - Delta_s - actually starts being
  defined)
- goes to -oo at some positive value
- comes back from oo (where the y - effective price - actually starts being
  defined)
- decreases
- reaches a minimum
- increases
- goes to oo

## substituting solution for d_pos back:

In [53]:
sol1_subs = sol1.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs })
  .simplify();
large(sol1_subs);
large(
  Eq(Delta_b, deltaByDelta["b"].subs({ Delta_s: sol1_subs.rhs }).simplify()),
);

<IPython.core.display.Math object>

<IPython.core.display.Math object>

# more fine-grained rounding analysis

- required, as the worst case one above is undefined for some constants (d <= 0)

In [54]:
eps_b, eps_s = symbols('epsilon_b epsilon_s', positive=True, real=True)
eff_worse = (Delta_s + eps_s) / (Delta_b - eps_b) # worse case resulting from rounding Deltas
eff_worse

(Delta_s + epsilon_s)/(Delta_b - epsilon_b)

In [55]:
eff_worse_Delta_s = eff_worse.subs({ Delta_b: deltaByDelta["b"] });
large(eff_worse_Delta_s);
eff_worse_Delta_s = eff_worse_Delta_s.simplify();
large(eff_worse_Delta_s);

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [56]:
large(expand(eff_worse_Delta_s).simplify());

<IPython.core.display.Math object>

In [57]:
numerator = Delta_s ** 2 * (w_b + w_s) +
  Delta_s * (l_s * w_s + eps_s * w_b + eps_s * w_s) + eps_s * l_s * w_s;
denominator = Delta_s * (l_b * w_b - eps_b * w_b - eps_b * w_s) -
  eps_b * l_s * w_s;
f = numerator / denominator;
large(f);
Eq(f, eff_worse_Delta_s).simplify();

<IPython.core.display.Math object>

True

In [58]:
a_subs = w_b + w_s;
b_subs = l_s * w_s + eps_s * w_b + eps_s * w_s;
c_subs = l_s * w_s;
d_subs = l_b * w_b - eps_b * w_b - eps_b * w_s;

f1 = f.subs({ a_subs: a, b_subs: b, c_subs: c, d_subs: d_pos });
display(f1);

(Delta_s**2*a + Delta_s*b + c*epsilon_s)/(Delta_s*d_pos - c*epsilon_b)

In [59]:
d_f1 = f1.diff(Delta_s).simplify();
display(d_f1);

(-d_pos*(Delta_s**2*a + Delta_s*b + c*epsilon_s) + (2*Delta_s*a + b)*(Delta_s*d_pos - c*epsilon_b))/(Delta_s*d_pos - c*epsilon_b)**2

In [60]:
sols_f1 = solveset(d_f1, Delta_s);
display(sols_f1);

{c*epsilon_b/d_pos - sqrt(c)*sqrt(a*c*epsilon_b**2 + b*d_pos*epsilon_b + d_pos**2*epsilon_s)/(sqrt(a)*d_pos), c*epsilon_b/d_pos + sqrt(c)*sqrt(a*c*epsilon_b**2 + b*d_pos*epsilon_b + d_pos**2*epsilon_s)/(sqrt(a)*d_pos)}

In [61]:
sol1 = Eq(Delta_s, sols_f1.args[0]);
display(sol1);
print("\t-> everything positive -> solution positive -> VALID");
sol2 = Eq(Delta_s, sols_f1.args[1]);
display(sol2);
print(
  "\t-> needs to be investigated further (spoiler: it'll turn out to be a maximum anyways, so we can skip it)",
);

Eq(Delta_s, c*epsilon_b/d_pos + sqrt(c)*sqrt(a*c*epsilon_b**2 + b*d_pos*epsilon_b + d_pos**2*epsilon_s)/(sqrt(a)*d_pos))

	-> everything positive -> solution positive -> VALID


Eq(Delta_s, c*epsilon_b/d_pos - sqrt(c)*sqrt(a*c*epsilon_b**2 + b*d_pos*epsilon_b + d_pos**2*epsilon_s)/(sqrt(a)*d_pos))

	-> needs to be investigated further (spoiler: it'll turn out to be a maximum anyways, so we can skip it)


What happens if d is zero or negative?

- zero d:

In [62]:
display(Eq(d_subs, 0))
display(f) # recall: this is our reformulation of the effective price by Delta_s, worse case scenario
display(f.subs({d_subs: 0}).simplify())
display(f.subs({d_subs: 0}).simplify().expand())

Eq(-epsilon_b*w_b - epsilon_b*w_s + l_b*w_b, 0)

(Delta_s**2*(w_b + w_s) + Delta_s*(epsilon_s*w_b + epsilon_s*w_s + l_s*w_s) + epsilon_s*l_s*w_s)/(Delta_s*(-epsilon_b*w_b - epsilon_b*w_s + l_b*w_b) - epsilon_b*l_s*w_s)

(Delta_s**2*(-w_b - w_s) - Delta_s*(epsilon_s*w_b + epsilon_s*w_s + l_s*w_s) - epsilon_s*l_s*w_s)/(epsilon_b*l_s*w_s)

-Delta_s**2*w_b/(epsilon_b*l_s*w_s) - Delta_s**2/(epsilon_b*l_s) - Delta_s*epsilon_s*w_b/(epsilon_b*l_s*w_s) - Delta_s*epsilon_s/(epsilon_b*l_s) - Delta_s/epsilon_b - epsilon_s/epsilon_b

-> given that all summands for the effective price are negative, this solution
is indeed invalid

- negative d:

In [63]:
display(Lt(d_subs, 0));

-epsilon_b*w_b - epsilon_b*w_s + l_b*w_b < 0

-> denominator is negative, numerator is positive -> effective price is negative
-> invalid

=> there are no solutions besides the ones for d_pos

(up to this point we are analogous to the worst case where epsilons = 1)

In [64]:
large(Gt(d_subs, 0));
large(Gt(l_b - 1, eps_b * w_s / w_b));
large(Lt(eps_b, (l_b - 1) * w_b / w_s));

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### -> the difference is: We now have an upper limit on the rounding error on the buying-amount

- curiously enough not on eps_s

### analysing second derivative to determine minimum-status of solutions:

In [65]:
dd_f1 = d_f1.diff(Delta_s).simplify();
large(dd_f1);

<IPython.core.display.Math object>

In [66]:
dd_sol1 = dd_f1.subs({Delta_s: sol1.rhs}).simplify()
large(dd_sol1)
if Eq(dd_sol1, 0) == False: print("-> nonzero -> extremum")
if (dd_sol1.is_positive == True): print("-> positive -> minimum")
dd_sol2 = dd_f1.subs({Delta_s: sol2.rhs}).simplify()
large(dd_sol2)
if Eq(dd_sol2, 0) == False: print("-> nonzero -> extremum")
if (dd_sol2.is_negative == True): print("-> negative -> maximum")

<IPython.core.display.Math object>

-> nonzero -> extremum
-> positive -> minimum


<IPython.core.display.Math object>

-> nonzero -> extremum
-> negative -> maximum


### investigating limits:

In [67]:
display(f1.subs(Delta_s, 0));
display(limit(f1, Delta_s, oo));
display(limit(f1, Delta_s, -oo));

-epsilon_s/epsilon_b

oo

-oo

In [68]:
f1_zero = solveset(f1, Delta_s);
display(f1_zero);
print("crosses x-axis at:");
display(f1_zero.args[0].args[0].simplify());
display(
  f1_zero.args[0].args[0].subs({
    a: a_subs,
    b: b_subs,
    c: c_subs,
    d_pos: d_subs,
  }).simplify(),
);
print("and:");
display(f1_zero.args[0].args[1].simplify());
display(
  f1_zero.args[0].args[1].subs({
    a: a_subs,
    b: b_subs,
    c: c_subs,
    d_pos: d_subs,
  }).simplify(),
);

Complement({-b/(2*a) - sqrt(-4*a*c*epsilon_s + b**2)/(2*a), -b/(2*a) + sqrt(-4*a*c*epsilon_s + b**2)/(2*a)}, {c*epsilon_b/d_pos})

crosses x-axis at:


(-b - sqrt(-4*a*c*epsilon_s + b**2))/(2*a)

(-epsilon_s*w_b - epsilon_s*w_s - l_s*w_s - sqrt(-4*epsilon_s*l_s*w_s*(w_b + w_s) + (epsilon_s*w_b + epsilon_s*w_s + l_s*w_s)**2))/(2*(w_b + w_s))

and:


(-b + sqrt(-4*a*c*epsilon_s + b**2))/(2*a)

(-epsilon_s*w_b - epsilon_s*w_s - l_s*w_s + sqrt(-4*epsilon_s*l_s*w_s*(w_b + w_s) + (epsilon_s*w_b + epsilon_s*w_s + l_s*w_s)**2))/(2*(w_b + w_s))

In [69]:
print("singularity at:");
f1_ndef = f1_zero.args[1].args[0];
display(f1_ndef);
display(f1_ndef.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs }));
print("singularity-limit from below:");
display(limit(f1, Delta_s, f1_ndef, dir = "-"));
print("singularity-limit from above:");
display(limit(f1, Delta_s, f1_ndef, dir = "+"));

singularity at:


c*epsilon_b/d_pos

epsilon_b*l_s*w_s/(-epsilon_b*w_b - epsilon_b*w_s + l_b*w_b)

singularity-limit from below:


-oo

singularity-limit from above:


oo

In [70]:
print("maximum at:");
display(sol2);
display(
  sol2.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs }).simplify(),
);

maximum at:


Eq(Delta_s, c*epsilon_b/d_pos - sqrt(c)*sqrt(a*c*epsilon_b**2 + b*d_pos*epsilon_b + d_pos**2*epsilon_s)/(sqrt(a)*d_pos))

Eq(Delta_s, (-epsilon_b*l_s*w_s*sqrt(w_b + w_s) + sqrt(l_b)*sqrt(l_s)*sqrt(w_b)*sqrt(w_s)*sqrt(-epsilon_b*epsilon_s*w_b - epsilon_b*epsilon_s*w_s + epsilon_b*l_s*w_s + epsilon_s*l_b*w_b))/(sqrt(w_b + w_s)*(epsilon_b*w_b + epsilon_b*w_s - l_b*w_b)))

-> we have a function that

- comes from -oo
- increases
- crosses the x-axis from below at some negative x (see equation above, which
  has only negative summands)
- reaches a maximum (which needs to happen between the two crossings of the
  x-axis)
- decreases
- crosses the x-axis from above at some negative x (we know this because we know
  that it crosses the y-axis at -eps_s/eps_b)
- crosses the y-axis at -eps_s/eps_b (where the x - Delta_s - actually starts
  being defined)
- goes to -oo at some positive value
- comes back from oo (where the y - effective price - actually starts being
  defined)
- decreases
- reaches a minimum
- increases
- goes to oo

-> except for the more fine-grained information on the y-axis-crossing
(-eps_s/eps_b instead of -1), this is the same behaviour

- both epsilons affect the solution
- only eps_b has any constraints:
  - eps_b < (l_b - 1) * w_b / w_s

## substituting solution for d_pos back:

In [71]:
sol1_subs = sol1.subs({ a: a_subs, b: b_subs, c: c_subs, d_pos: d_subs })
  .simplify();
large(sol1_subs);
large(
  Eq(Delta_b, deltaByDelta["b"].subs({ Delta_s: sol1_subs.rhs }).simplify()),
);

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [72]:
#sol1_subs = sol1.subs({
  a: a_subs,
  b: b_subs,
  c: c_subs,
  d_pos: d_subs,
  eps_s: 1,
}).simplify();
#large(sol1_subs);
#large(
  Eq(Delta_b, deltaByDelta["b"].subs({ Delta_s: sol1_subs.rhs }).simplify()),
);

## what this means for the algorithm

- set eps_s = 1 (already integrated in the equation above)
- set eps_b = (l_b - 1) * w_b / w_s
- if eps_b >= 1, use simplified equation from worst case analysis
- otherwise use the equation above
- calculate Deltas for this worst case
- ???

## alternative

- we now have a closed-form solution given the rounding errors
- problem of course is that we don't know them in advance
- we can't simply use the best or the worst rounding errors because both might
  be incorrect
- we want a solution where the actual rounding-errors are close to our epsilons:
  - Delta_b - floor(Delta_b) ~ eps_b
  - ceil(Delta_s) - Delta_s ~ eps_s
- this feels like a good-ish problem formulation for numerical methods
- we are still not taking the spot-exp-rounding into account

## could we have gotten the same result by simply wiggling the deltas we already got in the typescript?

- we can guess from our previous algorithm (which found the best solution almost
  all the time):
  - often we had to increment minSelling many times before an optimal solution
    was found
    - ### -> wiggling the deltas once won't do the trick
    - ### -> we might even need epsilons > 1, but we are not taking the spot-exp-rounding-effects into account here
  - once one was found, increasing them further, even 1000x, didn't improve
    things
  - given that for each minSelling, the deltas were already maximized for the
    given exps/spots, we conclude that those improvements each had to have
    worsened prices

## next steps

- can we make any statements about the shape of the ultimate optimization
  problem, given that we likely observed some concave-ish shape?
- can we find a good formulation that does include the
  spot-exp-rounding-effects?
  - maybe expressing things again in terms of exps?
    - what then?
- can we find some good ways to express the rounding differentiably?
  - i.e. some error-term?
  - i.e. some gradual rounding/relaxation variable?
- can we quantify the change in spot/delta/effectivePrice implied by a single
  whole exponent-step?
  - it will always worsen the price
  - it will always increase the delta of the asset
  - it will increase the delta of the other asset, if it was the limiting factor
  - otherwise it will necessitate an increase in the other asset's delta/spot
- if we could express what constitutes an optimum for a discrete solution in
  equality-form, we could use KKT/sympy again
  - in inequality-form we can say that all surrounding points must be worse, or
    a variant thereof
  - in the continuous case we set the derivative to zero
  -

## why are we doing this again?

- two places where we might need this: swapping and simulation in
  liquidity-provision
- in swapping we are already looking at the biggest possible swap and all
  subswaps
  - no we are not, we are looking at the best swap for a user-defined minimum
    amount and all subswaps
    - this also means we are probably not displaying single swaps worse than
      that; requiring the user to increase minAmount. Not sure about all of this
      - -> because for unlimited subsequents we would never want to do that
      - still need a way to "squish" number of subsequents down, increasing the
        per-swap-amount past the optimum
  - but we could; this would definitely find us the best swap
  - might be too expensive though
  - we might make use of the swapfinding3 for that
  - another argument for exhaustive search is that we can display the
    zigzag-line to the user, helping them to find the optimal swap-amounts
    - alternatively suggest optimum amounts for the pair as well as closest
      optima to user-defined amount (or smth like that)
  - what are the implications for the choice of multiple subsequent swaps vs.
    differently sized ones?
    - if we have time, we can list all possible combinations
    - generally speaking though, assuming we don't mind multiple transactions,
      it would be best to find the optimal single swap for each
    - we could add number of subsequent transactions as another parameter in our
      optimization-problem
  - counterpoint/argument for finding optimal single swaps:
    - if we are not limited in the amount of subsequent swaps
    - if we are not interested in the zigzag-line
    - if we are in the simulation in liquidity-provision

In [73]:
raise Exception("stop here")

Exception: stop here

(below: some variant to deltaByDelta: spotBySpot)

In [ ]:
def spotBySpot_(asset, s, l, v, b, w, j, e, Delta, a, min):
  f = solveset(a0_eq.subs({Delta_b: deltaBySpot['b'], Delta_s: deltaBySpot['s']}), s)
  f = list(f)
  assert len(f) == 1
  f = f[0]
  if simp:
    f = simplify(f)
  return f

spotBySpot = {asset: spotBySpot_(asset, *all_symbols[asset]) for asset in assets}
spotBySpot['b']

l_b*s_s*w_b*w_s/(-l_s*w_b*w_s + s_s*w_b + s_s*w_s)

In [ ]:
spotBySpot["s"];

l_s*s_b*w_b*w_s/(-l_b*w_b*w_s + s_b*w_b + s_b*w_s)

all four solutions are the same irrespective of delta_b_negative (continuous
case, and not accounting for flipped indices, of course)

In [ ]:
denominator_s_s = spotBySpot["s"].args[-1].args[0];
display(denominator_s_s);
ndef_s_b = solveset(denominator_s_s, s_b);
assert(len(ndef_s_b) == 1);
ndef_s_b = ndef_s_b.args[0];
ndef_s_b;

-l_b*w_b*w_s + s_b*w_b + s_b*w_s

l_b*w_b*w_s/(w_b + w_s)

In [ ]:
denominator_s_b = spotBySpot["b"].args[-1].args[0];
display(denominator_s_b);
ndef_s_s = solveset(denominator_s_b, s_s);
assert(len(ndef_s_s) == 1);
ndef_s_s = ndef_s_s.args[0];
ndef_s_s;

-l_s*w_b*w_s + s_s*w_b + s_s*w_s

l_s*w_b*w_s/(w_b + w_s)

In [ ]:
display(ndef_Delta_b);
display(ndef_Delta_s);

l_b*w_b/(w_b + w_s)

-l_s*w_s/(w_b + w_s)

In [ ]:
display(ndef_Delta_b * -w_s);
display(ndef_Delta_s * -w_b);

-l_b*w_b*w_s/(w_b + w_s)

l_s*w_b*w_s/(w_b + w_s)

- the implied limits on spot prices look similar to those on Delta_b
- this similarity is encapsulated by negation and multiplication by the other
  asset's weight, which is odd, but reminds us a bit of the amm price
  calculation

what happens if we use those invalid spot prices in our other equations? Do they
break or not?

In [ ]:
ndef_s_s = solveset(spotBySpot["b"].args[-1].args[0], s_s);
assert(len(ndef_s_s) == 1);
ndef_s_s = ndef_s_s.args[0];
ndef_s_s;

l_s*w_b*w_s/(w_b + w_s)

In [ ]:
a0_ndef_spots = a0_eq.subs({ s_b: ndef_s_b, s_s: ndef_s_s });
a0_ndef_spots;

Eq(Delta_b*l_s*w_b*w_s/(w_b + w_s), Delta_s*l_b*w_b*w_s/(w_b + w_s))

In [ ]:
solveset(a0_ndef_spots, Delta_b);

{Delta_s*l_b/l_s}

In [ ]:
solveset(a0_eq, Delta_s);

{Delta_b*s_s/s_b}

In [ ]:
solveset(a0_ndef_spots, Delta_s);

{Delta_b*l_s/l_b}